A extração dos itens foi "fatiada" em seções e categorias para facilitar e permitir que o processo seja realizado corretamente. Assim, são mais de um código, separados em categorias (como "Infantil" ou "Medicamentos", por exemplo), e as categorias são divididas em subdivisões ("Marcas", por exemplo), afim de dividir volumes grandes de produtos (já que grandes volumes podem demorar muito tempo para rodar, além de aumentar as chances de encontrar algum problema nas páginas e dificultar o encontro do mesmo).
Foi observado que os links mudam conforme as categorias, páginas carregadas e filtros aplicados.

O código abaixo é capaz de extrair os nomes e preços dos produtos a partir do link, porém se limita aos produtos já carregados (15, no caso das páginas de categorias). Assim, é um passo inicial mas não é o que buscamos.

In [3]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.drogariasminasmais.com.br/medicamentos' 

# Realizando o pedido da página
response = requests.get(url)

# Criando o objeto BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

product_name_elements = soup.find_all('span', class_='vtex-product-summary-2-x-productBrand')

price_elements = soup.find_all('span', class_='vtex-product-price-1-x-sellingPriceValue')

if len(product_name_elements) == len(price_elements):
    for name, price in zip(product_name_elements, price_elements):
        
        product_name = name.get_text(strip=True)
        price_value = price.get_text(strip=True)
        

        print(f'Produto: {product_name} - Preço: {price_value}')
else:
    print("Número de produtos e preços não coincidem.")
    
print(f"Total de produtos coletados: {len(product_name_elements)}")



Produto: Centrum Pó Energia Sachê Sabor Guaraná - 5g - Preço: R$3,99
Produto: Pantoprazol 20mg com 28 Comprimidos - Cimed Genérico - Preço: R$6,14
Produto: Pantoprazol 20mg com 28 Comprimidos - Biosintetica Genérico - Preço: R$4,12
Produto: Desogestrel 0,75mg com 84 Comprimidos - Eurofarma Genérico - Preço: R$28,19
Produto: Desogestrel 0,75mg com 84 Comprimidos - Germed Genérico - Preço: R$24,46
Produto: Adeforte com 30 Cápsulas  - Gross - Preço: R$42,49
Produto: Font D 2.000UI Gotas com 10ml - União Química - Preço: R$17,66
Produto: Peagesic 300mg com 30 Comprimidos - Eurofarma - Preço: R$209,49
Produto: Dorflex com 36 Comprimidos - Sanofi - Preço: R$16,90
Produto: Dipirona Sódica Sabor Abacaxi Gotasl  20ml - Neo Química Genérico - Preço: R$5,49
Produto: Vitamina D3 7.000UI com 30 Cápsulas - Neo Química Genérico - Preço: R$10,00
Produto: Aerodini Spray com 200 Doses - Teuto - Preço: R$26,55
Produto: Colírio Teuto - 20ml - Preço: R$9,99
Produto: Leite de Magnésia Magmilk Tradicional co

Esse código (abaixo) é capaz de pesquisar as páginas seguintes e carregar mais produtos, tendo um limite de produtos (que pode ser alterado) e a quantidade de produtos por página. Porém, devido a grande quantidade de alguns dados, ele não carrega algumas categrorias. No caso de "Nossas Vitaminas", por exemplo, o código é funcional. 

In [6]:
import requests
from bs4 import BeautifulSoup
import time

# Adicionando um pequeno atraso entre as requisições
time.sleep(1)  # Atraso de 1 segundo

# URL base do site
base_url = 'https://www.drogariasminasmais.com.br/higiene-pessoal'

# Número total de produtos e produtos por página
total_products = 1093
products_per_page = 15

# Calculando o número total de páginas
total_pages = total_products // products_per_page + (1 if total_products % products_per_page != 0 else 0)

all_products = []

# Loop para percorrer todas as páginas
for page_num in range(1, total_pages + 1):
    # URL da página com a numeração
    url = f'{base_url}?page={page_num}'
    
    # Realizando a requisição para a página
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Erro ao acessar a página {page_num}: {response.status_code}")
        continue
    
    # Usando BeautifulSoup para parsear o conteúdo
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Encontrando os elementos dos produtos na página
    product_name_elements = soup.find_all('span', class_='vtex-product-summary-2-x-productBrand')
    price_elements = soup.find_all('span', class_='vtex-product-price-1-x-sellingPriceValue')

    # Verificando se o número de produtos e preços coincide
    if len(product_name_elements) == len(price_elements):
        for name, price in zip(product_name_elements, price_elements):
            product_name = name.get_text(strip=True)
            price_value = price.get_text(strip=True)
            all_products.append({'name': product_name, 'price': price_value})
            print(f'Produto: {product_name} - Preço: {price_value}')
           
    else:
        print(f"Erro na página {page_num}: o número de produtos e preços não coincide.")

print(f"Total de produtos coletados: {len(all_products)}")


Produto: Aparelho de Barbear Gillette Corpo - Leve 4 Pague 3 - Preço: R$32,99
Produto: Absorvente Interno Tradicional Super Isacare - 8 Uni - Preço: R$7,99
Produto: Absorvente Interno Tradicional Médio Isacare - 8 Uni - Preço: R$7,99
Produto: Absorvente Isacare Interno Super - 16 Uni - Preço: R$16,49
Produto: Absorvente Isacare Interno Médio - 16 Uni - Preço: R$16,49
Produto: Kit Escovas Dentais Kess + Protetor de Cerdas - Preço: R$16,49
Produto: Escova Dental Kess Média Leve 3 Pague 2 - 2564 - Preço: R$6,99
Produto: Escova Dental Kess Pro Extra Macia - 2 Uni - Preço: R$18,99
Produto: Escova Dental Kids Kess Basic - 3 Uni - Preço: R$6,99
Produto: Escova Dental Kess Pro - Extra Macia - Preço: R$11,99
Produto: Escova Dental Kess Pro Extra Macia - 3 Uni - Preço: R$24,99
Produto: Kit Escovas Dental Kess Combo Supra - 4 Uni - Preço: R$9,99
Produto: Carmed Enxaguante Bucal Fini Minhocas - 250ml - Preço: R$19,49
Produto: Desodorante Creme Dove Sérum Vitamina E - 50g - Preço: R$16,49
Produto: 

Esse nao roda tudo. Nao tem qtd de produtos.

In [6]:
import requests
from bs4 import BeautifulSoup
import time

# Atraso entre as requisições para evitar sobrecarregar o servidor
time.sleep(1)

# URL base do site
base_url = 'https://www.drogariasminasmais.com.br/infantil'

# Número de produtos por página esperado
products_per_page = 15

# Variáveis de controle
all_products = []
page_num = 1  # Página inicial

while True:
    # Montando a URL da página com a numeração
    url = f'{base_url}?page={page_num}'
    
    # Realizando a requisição para a página
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Erro ao acessar a página {page_num}: {response.status_code}")
        break  # Interrompe o loop caso haja erro na requisição
    
    # Usando BeautifulSoup para parsear o conteúdo da página
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Encontrando os elementos dos produtos na página
    product_name_elements = soup.find_all('span', class_='vtex-product-summary-2-x-productBrand')
    price_elements = soup.find_all('span', class_='vtex-product-price-1-x-sellingPriceValue')

    # Verificando se há produtos na página
    if len(product_name_elements) == 0:
        print("Nenhum produto encontrado, finalizando a coleta.")
        break  # Se não houver produtos, termina a coleta
    
    # Verificando se o número de produtos e preços coincide
    if len(product_name_elements) == len(price_elements):
        for name, price in zip(product_name_elements, price_elements):
            product_name = name.get_text(strip=True)
            price_value = price.get_text(strip=True)
            all_products.append({'name': product_name, 'price': price_value})
            print(f'Produto: {product_name} - Preço: {price_value}')
        
        page_num += 1  # Aumenta o número da página para continuar a busca
    else:
        print(f"Erro na página {page_num}: o número de produtos e preços não coincide.")
        break  # Interrompe o loop caso haja erro na estrutura da página

# Exibe o total de produtos coletados
print(f"Total de produtos coletados: {len(all_products)}")


Erro ao acessar a API na página 1: 400
Total de produtos coletados: 0
